In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import lightgbm as LGB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
import featurewiz

Imported featurewiz: advanced feature engg and selection library. Version=0.0.42
output = featurewiz(dataname, target, corr_limit=0.70,
                    verbose=2, sep=',', header=0, test_data='',
                    feature_engg='', category_encoders='')
Create new features via 'feature_engg' flag : ['interactions','groupby','target']
                                


In [18]:
df_clean = pd.read_csv('/Users/williamjames/Desktop/CSCI/487/final_data.csv')
df_clean = df_clean.drop(columns=['Unnamed: 0'])
df_home = df_clean.loc[(df_clean['location']==2)]
X = df_home.drop(columns=['win','location'])
y = df_home['win']




Accuracy of logistic regression classifier on test set: 0.66


In [16]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    lgb = LGB.LGBMClassifier()
    sfs1 = sfs(lgb, k_features=k, forward=True, verbose=0, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    lgb =LGB.LGBMClassifier()
    X_train, X_test, y_train, y_test = train_test_split(X[feat_names],y, test_size=.2)
    lgb.fit(X_train,y_train)
    y_pred = lgb.predict(X_test)
    n_scores = cross_val_score(lgb, X[feat_names], y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    k_features.append(k)
    direction.append('forwards')
    features.append(feat_names)
    score.append(np.mean(n_scores))
forward = list(zip(k_features,direction,features,score))

In [ ]:
k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    lgb = LGB.LGBMClassifier()
    sfs1 = sfs(lgb, k_features=k, forward=False, verbose=0, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    lgb =LGB.LGBMClassifier()
    X_train, X_test, y_train, y_test = train_test_split(X[feat_names],y, test_size=.2)
    lgb.fit(X_train,y_train)
    y_pred = lgb.predict(X_test)
    n_scores = cross_val_score(lgb, X[feat_names], y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(k)
    direction.append('backwards')
    features.append(feat_names)
    score.append(np.mean(n_scores))
backwards = list(zip(k_features,direction,features,score))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
k_features = []
direction = []
scores = []
feats = []
accuracy = []
for i in range(5,19):
    rfe = RFE(estimator=LGB.LGBMClassifier(), n_features_to_select=i)
    rfe.fit(X,y)
    lgb = LGB.LGBMClassifier()
    support = rfe.support_
    feature_names = X_train.columns
    rfe_feats = feature_names[support]
    # fit the model on all available data
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    lgb.fit(X_train,y_train)
    n_scores = cross_val_score(lgb, X[rfe_feats], y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(k)
    direction.append('rfe')
    features.append(feat_names)
    score.append(np.mean(n_scores))
forward = list(zip(k_features,direction,features,score))

In [ ]:
from sklearn.decomposition import PCA

k_features = []
direction = []
scores = []
feats = []

for i in range(1,20):
    pca = PCA(n_components=i)
    X_pca = pca.fit_transform(X)

    # Get the transformed dataset
    X_pca = pd.DataFrame(X_pca)
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.20, 
                                                            shuffle=True, random_state=2)


    clf = LGB.LGBMClassifier()

    # Train the model
    clf.fit(X_train_pca, y_train)

    # Make predictions
    y_pred = clf.predict(X_test_pca) # Predictions
    y_true = y_test # True values

    # Measure accuracy
    from sklearn.metrics import accuracy_score
    import numpy as np
    n_scores = cross_val_score(clf, X_pca, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(k)
    direction.append('pca')
    features.append('all')
    score.append(np.mean(n_scores))
pca_data = list(zip(k_features,direction,feats,scores))

In [23]:
bwd = pd.DataFrame(data=backwards[:],columns=['k_features','direction','features','score'])
fwd = pd.DataFrame(data=forward[:],columns=['k_features','direction','features','score'])
rfe_results = pd.DataFrame(data=rfe_res[:],columns=['k_features','direction','features','score'])
pca_results = pd.DataFrame(data=pca_data[:],columns=['k_features','direction','features','score'])
results = pd.concat([bwd,fwd,rfe_results,pca_results])

In [29]:
results.sort_values('score',ascending=False)

,k_features,direction,features,score
14,19,forwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.674450
9,10,pca,all,0.672654
8,9,pca,all,0.667265
16,17,pca,all,0.666367
14,15,pca,all,0.665469
...,...,...,...,...
11,16,backwards,"[HHI_assists, F_PTS, C_PTS, starter_PTS, start...",0.636731
0,5,rfe,"Index(['G_PTS', 'TO_rate', 'EFG', 'FT_rate', '...",0.634486
0,1,pca,all,0.633139
14,19,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.631792


In [27]:
from sklearn.model_selection import cross_val_score
fts = []
direction = []
k_features = []
accuracy = []
lgb = LGB.LGBMClassifier()
feats = ['opp_play_percent', 'play_percent', 'TO_rate', 'OREB_rate', 'FT_rate', 'opp_TO_rate', 'opp_OREB_rate', 'starter_MIN', 'C_PTS', 'F_PTS', 'opp_FT_rate']
X = df_home[feats]
y = df_home['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)
lgb.fit(X_train,y_train)
n_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
fts.append(feats)
k_features.append(len(feats))
direction.append('featurewiz')
accuracy.append(lgb.score(X_test,y_test))
fwiz = list(zip(k_features,direction,fts,accuracy))
fw_res = pd.DataFrame(data=fwiz[:],columns=['k_features','direction','features','score'])


0.65828468792097


In [28]:
results = pd.concat([results,fw_res])

In [30]:
results.to_csv('LGBModel_Results.csv',index=False)